In [1]:
#import libraries.
import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
from tensorflow.python.keras.utils.data_utils import Sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import keras
from keras import regularizers
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import sounddevice as sd
import tensorflow as tf
from mir_eval import separation 
from pystoi.stoi import stoi 
import h5py
from keras.callbacks import LearningRateScheduler
from keras.layers.normalization import BatchNormalization
from keras.layers import Activation
import math
from sklearn.utils import shuffle
from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
#import libraries.
# import matplotlib.pyplot as plt
# from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
import time
import pickle
from keras import backend as K
from tensorflow.keras.models import load_model
# from keras.models import load_model

In [3]:
def reconstruct(wave,angle):
    recon1 = wave*np.cos(angle)+wave*np.sin(angle)*1j
#     recon = np.sqrt(np.power(10, wave))
#     recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=160, win_length=400, window='hann')
    return recon

In [4]:
data_series = str(4)

In [5]:
estimator_A = load_model('Models/Two_stage/trained_model19.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [51]:
# FF
h5f = h5py.File('TIMIT/Organized/concatenated/Test/Mixed_FF_test' + data_series + '.hdf5','r')
test_input = h5f[list(h5f.keys())[0]][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/clean_FF_test' + data_series + '.hdf5','r')
test_target = h5f[list(h5f.keys())[0]][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/clean_FF_test_phase' + data_series + '.hdf5','r')
angle_clean = h5f[list(h5f.keys())[0]][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/Mixed_FF_test_phase' + data_series + '.hdf5','r')
angle_mixed = h5f[list(h5f.keys())[0]][::]
h5f.close()

In [51]:
# MM
h5f = h5py.File('TIMIT/Organized/concatenated/Test/Mixed_MM_test' + data_series + '.hdf5','r')
test_input = h5f['mixed_MM_test'  + data_series][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/clean_MM_test' + data_series + '.hdf5','r')
test_target = h5f['clean_MM_test'  + data_series][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/clean_MM_test_phase' + data_series + '.hdf5','r')
angle_clean = h5f['clean_MM_test_phase'  + data_series][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/Mixed_MM_test_phase' + data_series + '.hdf5','r')
angle_mixed = h5f['Mixed_MM_test_phase'  + data_series][::]
h5f.close()

In [81]:
# FM
h5f = h5py.File('TIMIT/Organized/concatenated/Test/Mixed_FM_test' + data_series + '.hdf5','r')
test_input = h5f[list(h5f.keys())[0]][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/clean_FM_test' + data_series + '.hdf5','r')
test_target = h5f[list(h5f.keys())[0]][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/clean_FM_test_phase' + data_series + '.hdf5','r')
angle_clean = h5f[list(h5f.keys())[0]][:]
h5f.close()
h5f = h5py.File('TIMIT/Organized/concatenated/Test/Mixed_FM_test_phase' + data_series + '.hdf5','r')
angle_mixed = h5f[list(h5f.keys())[0]][::]
h5f.close()

In [25]:
test_input.shape

(18862, 257)

In [26]:
# test_input = sklearn.preprocessing.normalize(test_input, norm='l2', axis=1, copy=True, return_norm=False)
# Data1 = sklearn.preprocessing.normalize(Data1, norm='l2', axis=1, copy=True, return_norm=False)

In [113]:
test_input = preprocessing.normalize(test_input, norm='l2', axis=1, copy=True)
# test_target = (test_target-np.mean(test_target))/(np.max(test_target) - np.min(test_target))

In [114]:
print(test_input.shape)
print(test_target.shape)
print(angle_clean.shape)
print(angle_mixed.shape)

(514, 295522)
(514, 295522)
(257, 295522)
(257, 295522)


In [84]:
fft_samples = 257

In [115]:
# prediction1 = estimator_A.predict(test_input_normalized)
prediction1 = estimator_A.predict(test_input)

ValueError: Error when checking input: expected dense_21_input to have shape (257,) but got array with shape (514,)

In [86]:
speaker1 = np.multiply(test_input, prediction1[:,0:fft_samples])
speaker2 = np.multiply(test_input, prediction1[:,fft_samples:])

In [87]:
# prediction[1000]

In [88]:
s1 = reconstruct(speaker1, angle_mixed)
s2 = reconstruct(speaker2, angle_mixed)

In [89]:
clean_recon1 = reconstruct(test_target[:,0:fft_samples], angle_clean[:,0:fft_samples])
clean_recon2 = reconstruct(test_target[:,fft_samples:], angle_clean[:,fft_samples:])

In [90]:
mixed_recon = reconstruct(test_input[:], angle_mixed)

In [91]:
st_recon1 = stoi(s1, clean_recon1, 16000, extended=False)
st_recon2 = stoi(s2, clean_recon2, 16000, extended=False)

In [92]:
st_mixed = stoi(mixed_recon, clean_recon1, 16000, extended=False)

In [93]:
print('STOI_recon1=', st_recon1)
print('STOI_recon2=', st_recon2)
print('STOI_mixed=', st_mixed)

STOI_recon1= 0.408433205511285
STOI_recon2= 0.5260210703000598
STOI_mixed= 0.3243004405902902


In [94]:
sdr_recon1, sir_recon1, sar_recon1, perm_recon1 = separation.bss_eval_sources(s1, clean_recon1, compute_permutation=True)
sdr_recon2, sir_recon2, sar_recon2, perm_recon2 = separation.bss_eval_sources(s2, clean_recon2, compute_permutation=True)

In [95]:
sdr_mixed, sir_mixed, sar_mixed, perm_mixed = separation.bss_eval_sources(mixed_recon, clean_recon1, compute_permutation=True)

In [96]:
print('SDR_recon1=', sdr_recon1)
print('SDR_recon2=', sdr_recon2)
print('SDR_mixed=', sdr_mixed)

SDR_recon1= [-4.28108286]
SDR_recon2= [0.99858673]
SDR_mixed= [-4.78920035]


In [97]:
# start = int(len(s1)/20)
# stop = int(len(s1)/10)
# start = 0
# stop = 200000
start = 0
stop = 100000

In [68]:
sd.play(s1[start:stop]*2000, 16000)

In [69]:
sd.play(s2[start:stop]*2000, 16000)

In [80]:
sd.play(clean_recon1[start:stop]*10, 16000)

In [81]:
sd.play(mixed_recon[0:int(len(s1)/20)]*10, 16000)

In [58]:
test_target.shape

(18862, 514)

In [98]:
def loss_DNNC(y_true = test_target.T, y_pred = test_target.T, Lambda = 0.05):
    loss = K.sum(K.square(y_true - y_pred)) 
    - Lambda * K.sum(K.square(y_true[0:257,:] - y_pred[0:257,:]) + K.square(y_true[257:,:] - y_pred[257:,:]))
    return loss

In [99]:
estimator_C = load_model('Models/Two_stage/Model_C0.h5', 
                          custom_objects={'loss_DNNC': loss_DNNC})

In [100]:
sec_set = np.concatenate((speaker1 , speaker2), axis=1)

In [101]:
sec_set = preprocessing.normalize(sec_set, norm='l2', axis=1, copy=True)
test_target = preprocessing.normalize(test_target, norm='l2', axis=1, copy=True)

In [102]:
prediction2 = estimator_C.predict(sec_set)

In [103]:
prediction2.shape

(18634, 514)

In [104]:
s1_C = reconstruct(prediction2[:,0:257], angle_mixed)
s2_C = reconstruct(prediction2[:,257:], angle_mixed)

In [105]:
clean_recon1 = reconstruct(test_target[:,0:fft_samples], angle_clean[:,0:fft_samples])
clean_recon2 = reconstruct(test_target[:,fft_samples:], angle_clean[:,fft_samples:])

In [106]:
st_recon1 = stoi(s1_C, clean_recon1, 16000, extended=False)
st_recon2 = stoi(s2_C, clean_recon2, 16000, extended=False)

In [107]:
sdr_recon1, sir_recon1, sar_recon1, perm_recon1 = separation.bss_eval_sources(s1_C, clean_recon1, compute_permutation=True)
sdr_recon2, sir_recon2, sar_recon2, perm_recon2 = separation.bss_eval_sources(s2_C, clean_recon2, compute_permutation=True)

In [108]:
print('SDR_recon1=', sdr_recon1)
print('SDR_recon2=', sdr_recon2)
print('SDR_mixed=', sdr_mixed)

SDR_recon1= [-4.51841408]
SDR_recon2= [-2.20552222]
SDR_mixed= [-4.78920035]


In [109]:
print('STOI_recon1=', st_recon1)
print('STOI_recon2=', st_recon2)
print('STOI_mixed=', st_mixed)

STOI_recon1= 0.2992286684160478
STOI_recon2= 0.36715637272317214
STOI_mixed= 0.3243004405902902


In [50]:
sd.play(s1_C[start:stop]*2000, 16000)